In [1]:
import torch, mindspore
import numpy as np
from transformers.models.bart import modeling_bart as pt
import mindnlp.models.bart as m

/home/wangxingran/anaconda3/envs/mindspore_py37/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = r"/home/wangxingran/bart_migration/Bartckpt/facebook/bart-large-xsum"
config_path = f"{path}/config.json"
pytorch_model_path = f"{path}/pytorch_model.bin"

In [3]:
# init config
import json
with open(config_path, encoding='utf-8') as config:
    config = json.load(config)

pt_config = pt.BartConfig(**config)
pt_dict = torch.load(pytorch_model_path)
pt_model = pt.BartForConditionalGeneration(pt_config)
pt_model.load_state_dict(pt_dict, False)

_IncompatibleKeys(missing_keys=['final_logits_bias', 'lm_head.weight'], unexpected_keys=[])

In [4]:
pt_params = pt_model.state_dict()
for key in pt_params.keys():
    print(key)

final_logits_bias
model.shared.weight
model.encoder.embed_tokens.weight
model.encoder.embed_positions.weight
model.encoder.layers.0.self_attn.k_proj.weight
model.encoder.layers.0.self_attn.k_proj.bias
model.encoder.layers.0.self_attn.v_proj.weight
model.encoder.layers.0.self_attn.v_proj.bias
model.encoder.layers.0.self_attn.q_proj.weight
model.encoder.layers.0.self_attn.q_proj.bias
model.encoder.layers.0.self_attn.out_proj.weight
model.encoder.layers.0.self_attn.out_proj.bias
model.encoder.layers.0.self_attn_layer_norm.weight
model.encoder.layers.0.self_attn_layer_norm.bias
model.encoder.layers.0.fc1.weight
model.encoder.layers.0.fc1.bias
model.encoder.layers.0.fc2.weight
model.encoder.layers.0.fc2.bias
model.encoder.layers.0.final_layer_norm.weight
model.encoder.layers.0.final_layer_norm.bias
model.encoder.layers.1.self_attn.k_proj.weight
model.encoder.layers.1.self_attn.k_proj.bias
model.encoder.layers.1.self_attn.v_proj.weight
model.encoder.layers.1.self_attn.v_proj.bias
model.encod

In [5]:
# init config
ms_config = m.BartConfig(**config)
ms_model = m.BartForConditionalGeneration(ms_config)

In [6]:
# print ms_model parameters' name
for key, param in ms_model.parameters_and_names():
    print(param.name)

final_logits_bias
model.shared.embedding_table
model.encoder.embed_positions.embedding_table
model.encoder.layers.0.self_attn.k_proj.weight
model.encoder.layers.0.self_attn.k_proj.bias
model.encoder.layers.0.self_attn.v_proj.weight
model.encoder.layers.0.self_attn.v_proj.bias
model.encoder.layers.0.self_attn.q_proj.weight
model.encoder.layers.0.self_attn.q_proj.bias
model.encoder.layers.0.self_attn.out_proj.weight
model.encoder.layers.0.self_attn.out_proj.bias
model.encoder.layers.0.self_attn_layer_norm.gamma
model.encoder.layers.0.self_attn_layer_norm.beta
model.encoder.layers.0.fc1.weight
model.encoder.layers.0.fc1.bias
model.encoder.layers.0.fc2.weight
model.encoder.layers.0.fc2.bias
model.encoder.layers.0.final_layer_norm.gamma
model.encoder.layers.0.final_layer_norm.beta
model.encoder.layers.1.self_attn.k_proj.weight
model.encoder.layers.1.self_attn.k_proj.bias
model.encoder.layers.1.self_attn.v_proj.weight
model.encoder.layers.1.self_attn.v_proj.bias
model.encoder.layers.1.self_a

In [7]:
import logging
def torch_to_mindspore(pth_file, size:str=None):
    try:
        import torch
    except:
        raise ImportError(f"'import torch' failed, please install torch by "
                          f"`pip install torch` or instructions from 'https://pytorch.org'")

    size = "mindspore" if not size else size # rename ckpt

    from mindspore import Tensor
    from mindspore.train.serialization import save_checkpoint

    logging.info('Starting checkpoint conversion.')
    ms_ckpt = []
    state_dict = torch.load(pth_file, map_location=torch.device('cpu'))

    for k, v in ms_model.parameters_and_names():
        find_name = v.name.replace('embedding_table','weight').replace('gamma','weight').replace('beta','bias')
        if find_name not in state_dict:
            print(v.name)
        if find_name in pt.BartForConditionalGeneration._keys_to_ignore_on_load_missing:
            continue
        ms_ckpt.append({'name': v.name, 'data': Tensor(state_dict[find_name].numpy())})

    ms_ckpt_path = pth_file.replace('.bin','.ckpt')
    ms_ckpt_path = ms_ckpt_path.replace('pytorch',size)
    try:
        save_checkpoint(ms_ckpt, ms_ckpt_path)
    except:
        raise RuntimeError(f'Save checkpoint to {ms_ckpt_path} failed, please checkout the path.')

    return ms_ckpt_path

In [8]:
torch_to_mindspore(pytorch_model_path)

final_logits_bias
lm_head.weight


'/home/wangxingran/bart_migration/Bartckpt/facebook/bart-large-xsum/mindspore_model.ckpt'

In [9]:
config_path = f"{path}/config.json"
with open(config_path, encoding='utf-8') as config:
    config = json.load(config)
# init config
pt_config = pt.BartConfig(**config)
ms_config = m.BartConfig(**config)

# init model
pt_model = pt.BartForConditionalGeneration(pt_config)
ms_model = m.BartForConditionalGeneration(ms_config)

# load parameters
pt_dict = torch.load(f"{path}/pytorch_model.bin")
pt_model.load_state_dict(pt_dict, False) 

ms_dict = mindspore.load_checkpoint(f"{path}/mindspore_model.ckpt")
param_not_load = mindspore.load_param_into_net(ms_model, ms_dict)
print(f"Param_not_load:{param_not_load}")

# 手动copy
for k,v in ms_model.parameters_and_names():
    if v.name == 'lm_head.weight':
        v.set_data(mindspore.Tensor(pt_model.state_dict().get(v.name).detach().numpy()))

# set eval mode
pt_model.eval()
ms_model.set_train(False)

[WARNING] ME(10771:139799685108800,MainProcess):2023-04-07-19:36:05.558.288 [mindspore/train/serialization.py:1058] For 'load_param_into_net', 2 parameters in the 'net' are not loaded, because they are not in the 'parameter_dict', please check whether the network structure is consistent when training and loading checkpoint.
[WARNING] ME(10771:139799685108800,MainProcess):2023-04-07-19:36:05.644.669 [mindspore/train/serialization.py:1060] final_logits_bias is not loaded.
[WARNING] ME(10771:139799685108800,MainProcess):2023-04-07-19:36:05.646.144 [mindspore/train/serialization.py:1060] lm_head.weight is not loaded.


Param_not_load:(['final_logits_bias', 'lm_head.weight'], [])


BartForConditionalGeneration<
  (model): BartModel<
    (shared): Embedding<vocab_size=50264, embedding_size=1024, use_one_hot=False, embedding_table=Parameter (name=model.shared.embedding_table, shape=(50264, 1024), dtype=Float32, requires_grad=True), dtype=Float32, padding_idx=1>
    (encoder): BartEncoder<
      (embed_tokens): Embedding<vocab_size=50264, embedding_size=1024, use_one_hot=False, embedding_table=Parameter (name=model.shared.embedding_table, shape=(50264, 1024), dtype=Float32, requires_grad=True), dtype=Float32, padding_idx=1>
      (embed_positions): BartLearnedPositionalEmbedding<vocab_size=1026, embedding_size=1024, use_one_hot=False, embedding_table=Parameter (name=model.encoder.embed_positions.embedding_table, shape=(1026, 1024), dtype=Float32, requires_grad=True), dtype=Float32, padding_idx=None>
      (layers): CellList<
        (0): BartEncoderLayer<
          (self_attn): BartAttention<
            (k_proj): Dense<input_channels=1024, output_channels=1024, has

In [10]:
seq_len = 64
batch = 4
embed_dim = ms_config.d_model
tgt_len = 32
src_len = 32
encoder_attention_heads = ms_config.encoder_attention_heads

hidden_states = np.random.randn(batch,seq_len,embed_dim)
# input_ids = np.random.randint(128,None,(batch,seq_len))
input_ids = np.array([
                [71, 82, 18, 33, 46, 91, 2],
                [68, 34, 26, 58, 30, 82, 2],
                [5, 97, 17, 39, 94, 40, 2],
                [76, 83, 94, 25, 70, 78, 2],
                [87, 59, 41, 35, 48, 66, 2],
                [55, 13, 16, 58, 5, 2, 1],  # note padding
                [64, 27, 31, 51, 12, 75, 2],
                [52, 64, 86, 17, 83, 39, 2],
                [48, 61, 9, 24, 71, 82, 2],
                [26, 1, 60, 48, 22, 13, 2],
                [21, 5, 62, 28, 14, 76, 2],
                [45, 98, 37, 86, 59, 48, 2],
                [70, 70, 50, 9, 28, 0, 2],
                [70, 70, 50, 9, 28, 0, 2],
                [70, 70, 50, 9, 28, 0, 2],
                [70, 70, 50, 9, 28, 0, 2],
                [70, 70, 50, 9, 28, 0, 2],
                [70, 70, 50, 9, 28, 0, 2],
                [70, 70, 50, 9, 28, 0, 2],
                [70, 70, 50, 9, 28, 0, 2],
                [70, 70, 50, 9, 28, 0, 2],
                [70, 70, 50, 9, 28, 0, 2],
                [70, 70, 50, 9, 28, 0, 2],
                [70, 70, 50, 9, 28, 0, 2],
            ])
# attention_mask = np.random.randint(0,1,(batch,seq_len))
attention_mask = np.random.randn(batch,1,seq_len,seq_len)
head_mask = np.random.randint(0,1,(ms_config.encoder_layers,ms_config.encoder_attention_heads))
layer_head_mask = np.random.randn(encoder_attention_heads)

ms_hidden_states = mindspore.Tensor(hidden_states, dtype=mindspore.float32)
ms_input_ids = mindspore.Tensor(input_ids, dtype=mindspore.int64)
ms_attention_mask = mindspore.Tensor(attention_mask, dtype=mindspore.float32)
ms_head_mask = mindspore.Tensor(head_mask, dtype=mindspore.bool_)
ms_layer_head_mask = mindspore.Tensor(layer_head_mask, dtype=mindspore.float32)

pt_hidden_states = torch.tensor(hidden_states, dtype=torch.float)
pt_input_ids = torch.tensor(input_ids, dtype=torch.long)
pt_attention_mask = torch.tensor(attention_mask, dtype=torch.float)
pt_head_mask = torch.tensor(head_mask, dtype=torch.bool)
pt_layer_head_mask = torch.tensor(layer_head_mask, dtype=torch.float)

In [11]:
ms_out = ms_model(ms_input_ids)
pt_out = pt_model(pt_input_ids)

In [12]:
def judge(o1, o2, loss = 1e-3, prefix = '-'):
    prefix += '-'
    if (isinstance(o1, tuple)):
        assert len(o1) == len(o2)
        for i in range(len(o1)):
            judge(o1[i], o2[i], loss=loss, prefix=prefix)
    elif (isinstance(o1,mindspore.Tensor)):
        assert o1.shape == o2.shape
        print(f"{prefix}{np.allclose(o1.asnumpy(), o2.detach().numpy(), loss, loss)}")
    else:
        print(f"{type(o1)}-{type(o2)}:{o1==o2}")

judge(ms_out,pt_out)

---True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
-----True
---True


In [13]:
print(ms_out[0][0])
print(pt_out[0][0])

[[-9.931457    0.55640894  1.4794239  ...  0.37412488  0.4712171
   0.3184269 ]
 [-7.60198    -0.12448323  2.0910916  ... -0.3296257  -0.6810597
  -0.7874516 ]
 [-4.9174356   0.06131339  2.9956322  ... -0.33067703 -0.46984035
  -0.63954175]
 ...
 [-2.6086924   0.11346889  4.875864   ...  0.21028513  0.22378874
  -0.27078193]
 [ 1.4311728  -0.53205293 10.8360815  ... -0.55372524 -0.45854667
  -0.48365214]
 [-7.886387    0.08557969 12.221794   ... -0.63345397 -0.3903224
  -0.03797966]]
tensor([[-9.9315,  0.5564,  1.4794,  ...,  0.3741,  0.4712,  0.3184],
        [-7.6020, -0.1245,  2.0911,  ..., -0.3296, -0.6811, -0.7875],
        [-4.9174,  0.0613,  2.9956,  ..., -0.3307, -0.4698, -0.6395],
        ...,
        [-2.6087,  0.1135,  4.8759,  ...,  0.2103,  0.2238, -0.2708],
        [ 1.4312, -0.5321, 10.8361,  ..., -0.5537, -0.4585, -0.4837],
        [-7.8863,  0.0856, 12.2218,  ..., -0.6335, -0.3903, -0.0380]],
       grad_fn=<SelectBackward0>)
